In [13]:
pip install tokenizer==0.11

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 3.4.5 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement tokenizer==0.11 (from versions: 0.1.1, 0.1.2, 0.1.3, 0.1.10, 0.1.11, 0.1.12, 0.1.14, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.0.9, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0, 1.4.0, 1.4.1, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.1.0, 2.2.0, 2.3.0, 2.3.1, 2.4.0, 2.5.0, 3.0.0, 3.1.0, 3.1.1, 3.1.2, 3.2.0, 3.3.0, 3.3.1, 3.3.2, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.4.4)
ERROR: No matching distribution found for tokenizer==0.11


In [16]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

from sklearn_extra.cluster import KMedoids
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

import torch
import re


from transformers import pipeline

from typing import Union, List
from fastapi import FastAPI
from pydantic import BaseModel, ConfigDict, ValidationError
from pydantic_core import from_json
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os

def get_overlapped_chunks(text, chunk, overlap):
    return [text[a:a+chunk].strip() for a in range(0, len(text), chunk-overlap)]

In [18]:
hours_back = 8

section_number = 100

current_time = datetime.now()

start_time = current_time - timedelta(hours=hours_back)

valid_hours = []
for hour in range(1, hours_back + 1):  # 0부터 hours_back까지의 시간을 계산
    valid_hour = (start_time - timedelta(hours=hour)).strftime('%H')
    valid_hours.append(valid_hour)

file_patterns = [f"./crawling_data/{section_number}/_{hour}.pkl" for hour in valid_hours]

merged_documents = []
for file_pattern in file_patterns:
    if(os.path.isfile(file_pattern)):
        with open(file_pattern, 'rb') as file:
            merged_documents += pd.read_pickle(file)
            print(len(merged_documents))
            print(file_pattern)
print(f"문서의 개수: {len(merged_documents)} 개 입니다.")

print(file_patterns)

merged_documents.remove(merged_documents[12])
merged_documents.remove(merged_documents[29])
merged_documents.remove(merged_documents[27])
merged_documents.remove(merged_documents[38])
merged_documents.remove(merged_documents[35])


15
./crawling_data/100/_02.pkl
28
./crawling_data/100/_01.pkl
39
./crawling_data/100/_00.pkl
62
./crawling_data/100/_23.pkl
88
./crawling_data/100/_22.pkl
문서의 개수: 88 개 입니다.
['./crawling_data/100/_02.pkl', './crawling_data/100/_01.pkl', './crawling_data/100/_00.pkl', './crawling_data/100/_23.pkl', './crawling_data/100/_22.pkl', './crawling_data/100/_21.pkl', './crawling_data/100/_20.pkl', './crawling_data/100/_19.pkl']


In [24]:
data = []
for doc in merged_documents:
    data.append(doc.get("content"))

countvectorizer = CountVectorizer(stop_words=["\n", "기자", "뉴스"], ngram_range=(1,2), max_df = 0.4, min_df = 0.05, lowercase=True, max_features = 550)
data = TfidfTransformer().fit_transform(countvectorizer.fit_transform(data))

clst = KMedoids(n_clusters=4)
clst.fit(data)

medoid_indices = clst.medoid_indices_
print(f"중심으로 선정된 기사들의 index: {medoid_indices}")

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
summarizer = pipeline('summarization', model='gogamza/kobart-summarization')

briefings = []
for med in medoid_indices:
    text = merged_documents[med].get("content").replace("\n","").strip()
    re.sub('[^A-Za-z0-9가-힣]', '', text)

    while len(text) > 1024:
        outs = []   
        for chunk in get_overlapped_chunks(text, 256, 32):

            out = summarizer(chunk.strip(), max_length=64, min_length=8)
            outs.append(out[0]['summary_text'].strip())
        text = ' '.join(outs)

    briefings.append(text.strip())








중심으로 선정된 기사들의 index: [61 11 50 19]


Your max_length is set to 64, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 11. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 64, but you input_length is only 11. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [25]:
briefings

["남녀고용평등과 일·가정 양립지원법 개정안 등을 처리한 결과 육아휴직 기간을 현행 총 2년에서 부모별 1년 6개월씩 총 3년으로 확대하고 사용 기간 분할도 2회에서 3회로 늘렸다.       현재는 8세 이하 또는 초등학교 2학년 이하 자녀를 둔 남녀 근로자는 자녀  출산휴가 역시 출산일로부터 120일 이내에 3회 분할 사용이 가능하도록 규정을 완화했다.      현재는 출산 후 90일 이내에 10일 쓸 수 있다.        우선지원 대상기업(중소기업) 근로자에 대해서는 정부의 출산휴가 급여지원 기간도 4일에서 20일 육아기 근로 시간 단축 대상 자녀의 연령을 현행 8세에서 12세로 확대하는 내용이 포함되어 육아기 근로시간 단축에 쓸 수 있는 육아기 근로시간 단축을 최대 3년까지               육아기 근로시간 단축 기간은 현행 '12주 고용노동부는  근로시간 단축 기간은 현행 '12주 이내 36주 이후'에서 '12주 이내 32주 이후'로 확대하고 조기 진통, 다태아 임신 등 고위험 임신부는 의사의 진단에 따라 임신 전체 기간에 근로시간 단축을 사용할 수 있게 된다.      고용노동부는 하위 국가가 한부모가족에게 양육비를 먼저 지급한 뒤 비양육자로부터 나중에 받아내는 '양육비 선지급제' 도입 근거를 담은 양육비이행법 개정안도 이날 함께 처리됐다. 비양육자의 동의 없이도 금융정보를 포함한 소득·재산 조사를 가능하게 하는 등 선지급금 회수 관리체계를 강화하는 내용이 개정안에 포함됐다.",
 '국회 국회 국회 국회 국회 국회 국회 국회 국회 국회 국회 국회 국회 국회 본회의장에서 사전 합의로 추천한 국가인권위원 선출 표결 결과 선(先) 투표한 야당 추천 인사는 통과된 반면, 후(後) 투표한 여당 추천 인사는 부결된 반면, 후(後) 투표한 여당 추천 한석훈 국가인권위원 선출안이 국회에서 야당 주도로 부결된 부결된 부결된 투표에서도 여야 간 파열음은 우원식 의장에 직접 항의했다.뒤따라 나온 박찬대 민주당 원내대표는 "(인권위원 찬반을) 당론으로 정하지 않

In [95]:
load_dotenv(verbose=True)

model = "gpt-4o-mini";

prompt = """
-- 너에게 전날 밤에 있었던 뉴스들을 요약해서 들려주는 리포터 역할을 부여할게.
-- <description> 다음으로 4개의 기사를 각각 요약한 요약문들 4개가 주어질거야.
-- 그 요약문들을 모두 종합해서 자연스럽게 한국어 뉴스 브리핑을 만들어줘.
-- 친절한 말투로 부탁해.
-- 브리핑은 짧은 브리핑과 긴 브리핑 두가지로 만들어줘.
-- 짧은 브리핑은 300자, 긴 브리핑은 700자 정도로 만들어줘.
-- make sure there is no trailing comma

{
    "shortBriefing": "", 
    "longBriefing": ""
}

-- 위의 포맷을 채워서 반환해줘. 위의 포맷외에 아무런 내용도 담아서는 안돼.
<description>
"""

app = FastAPI(root_path="/ai/briefing")
client = AsyncOpenAI(api_key = os.getenv("API_KEY"))

class JSON_Briefing(BaseModel):
    text: str

class Briefing(BaseModel):
    problem: str
    choices: List[Choice]
    answer: int

class JSON_Briefing_Out(BaseModel):
    model_config = ConfigDict(strict=True)

    data: List[Quiz]


@app.post("/", response_model=JSON_Briefing_Out)
async def generate_briefing(json: JSON_Briefing):

chat_completion = await client.chat.completions.create(
    model= model,
    messages=[{"role": "user", "content": prompt + json.text}]
)

result = None
print(chat_completion.choices[0].message.content)

try:
    JSON_Briefing_Out.model_validate({'data':from_json(chat_completion.choices[0].message.content, allow_partial = True)})
    result = from_json(chat_completion.choices[0].message.content, allow_partial = True)
except ValidationError as e:
    print(e)

    return {"data" : result}